In [51]:
import numpy as np
import pandas as pd
import torch as th
from hierarchy_data import (
    CustomHierarchyData,
    normalize_data,
    unnormalize_data,
)
from models.fnpmodels import EmbedMetaAttenSeq, RegressionSepFNP, Corem
from utils import lag_dataset_2
from models.utils import float_tensor, long_tensor
from random_split import random_split

In [52]:
data_obj = CustomHierarchyData()

# Let's create dataset
full_data = data_obj.data
train_data_raw = full_data
train_means = np.mean(train_data_raw, axis=1)
train_std = np.std(train_data_raw, axis=1)
train_data = (train_data_raw - train_means[:, None]) / train_std[:, None]
num_stocks = train_data.shape[0]

dataset_raw = lag_dataset_2(train_data, data_obj.shapes, 1, 50)

class SeqDataset(th.utils.data.Dataset):
    def __init__(self, dataset):
        self.R, self.X, self.Y = dataset

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.R[idx], self.X[idx], self.Y[idx]

dataset = SeqDataset(dataset_raw)
train_dataset, val_dataset = random_split(dataset, [0.7, 0.3])

/media/harry/WB Passport1/Projects/profhit-fork/hierarchy_data/__init__.py:56: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  shapes = [df[str(date)].shape[0] for date in dates]


In [53]:
device = th.device('cuda')
# Let's create FNP model
encoder = EmbedMetaAttenSeq(
    dim_seq_in=1,
    num_metadata=len(data_obj.idx_dict),
    dim_metadata=1,
    dim_out=60,
    n_layers=2,
    bidirectional=True,
).to(device)
decoder = RegressionSepFNP(
    dim_x=60,
    dim_y=1,
    dim_h=60,
    n_layers=3,
    dim_u=60,
    dim_z=60,
    nodes=len(data_obj.idx_dict),
).to(device)
corem = Corem(nodes=len(data_obj.idx_dict), c=5.0, ).to(device)

In [54]:
encoder.load_state_dict(th.load('output/custom/encoder_train_32300_0.009501314722001553.pt'))
decoder.load_state_dict(th.load('output/custom/decoder_train_32300_0.009501314722001553.pt'))
corem.load_state_dict(th.load('output/custom/corem_train_32300_0.009501314722001553.pt'))

<All keys matched successfully>

In [55]:
r, x, y = train_dataset[2]
print(r.shape)

(258, 421)


In [56]:
ref_x = float_tensor(r[:, :, None]).to(device)
x = float_tensor(x[:, :, None]).to(device)
y = float_tensor(y[:, None]).to(device)
meta_x = long_tensor(np.arange(ref_x.shape[0]))
ref_out_x = encoder(ref_x, meta_x)
out_x = encoder(x, meta_x)
y_pred, mean_y, logstd_y, _ = decoder.predict(ref_out_x, out_x, sample=False)
y_pred, mean_y, logstd_y, _ = corem.predict(mean_y.squeeze(), logstd_y.squeeze(), sample=False)

In [57]:
print(y_pred.shape)
print(x.shape)

torch.Size([258, 1])
torch.Size([258, 316, 1])


In [58]:
gt_dir = (y[:, 0] - x[:, -1, 0]) > 0
pred_dir = (y_pred[:, 0] - x[:, -1, 0]) > 0

In [59]:
num_correct_dirs = th.count_nonzero(gt_dir == pred_dir)
print(num_correct_dirs / gt_dir.shape[0])
print(pred_dir)

tensor(0.4690, device='cuda:0')
tensor([ True,  True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True, False,  True,  True, False,  True, False, False, False,
         True,  True, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True,  True,
         True, False, False,  True,  True,  True, False, False,  True, False,
        False, False, False, False,  True,  True,  True, False, False, False,
         True,  True,  True,  True, False, False,  True,  True, False, False,
         True,  True,  True, False,  True,  True,  True, False, False,  True,
        False, False,  True,  True, False, False, False,  True, False, False,
         True,  True,  True, False,  True,  True,  True, False, False,  True,
         True,  True, False, False,  True, False,  True,  True,  True, False,
         True, False, False,  True, False,  True,  True, False,  True, False,
         True, False,  True,  Tr